<a href="https://colab.research.google.com/github/mitchelljphayes/COMP9444-group-assignment/blob/main/nx_base_FALCON_7Binstruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate datasets bitsandbytes einops wandb bert_score
import os
import logging
# import transformers
import bert_score
import pandas as pd
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from bert_score import score, BERTScorer
from datasets import load_dataset
import matplotlib.pyplot as plt
from matplotlib import rcParams

# transformers.tokenization_utils.logger.setLevel(logging.ERROR)
# transformers.configuration_utils.logger.setLevel(logging.ERROR)
# transformers.modeling_utils.logger.setLevel(logging.ERROR)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%matplotlib inline
bert_score.__version__

'0.3.12'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers
import torch

# model = "mrm8488/falcoder-7b"
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [4]:
from datasets import load_dataset

dataset_name = "mjphayes/machine_learning_questions"
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/508 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/128 [00:00<?, ? examples/s]

In [6]:
dataset.shape

(508, 3)

In [5]:
dataset[0]['question']

"What is 'natural language processing' (NLP) in machine learning?"

In [7]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
questions = [i['question'] for i in dataset]

# Batch processing
batch_size = 8  # Adjust as needed
batches = [questions[i:i+batch_size] for i in range(0, len(questions), batch_size)]

# Initialize an empty DataFrame to store results
result = pd.DataFrame(columns=['questions', 'answer', 'model_answer'])

# Process batches
for batch in batches:
    gens = pipeline(
        batch,
        max_length=200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    for i, gen in enumerate(gens):
        result.loc[len(result)] = [batch[i], dataset[i]['answer'], gen[0]['generated_text']]


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

In [ ]:
result_b = result.copy()

In [ ]:
result_b.to_csv('nx_7binstruct_base_trainset_output.csv',index=True)

STOP HERE

gen2

In [ ]:
dataset[0]['__index_level_0__']
result = pd.DataFrame(columns=['questions','answer','model_answer'])
for i in dataset:
  result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]

In [ ]:
result.shape

In [ ]:
result = pd.DataFrame(columns=['questions','answer','model_answer'])
# for i in dataset:
#   result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]

for i in dataset:
  prompt = i['question']
  gen = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
  )
  result.loc[i['__index_level_0__']] = [i['question'],i['answer'],gen[0]['generated_text']]
  # result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]



In [ ]:
result_b = result.copy()

In [ ]:
result

In [ ]:
result_b.to_csv('nx_7binstruct_base_trainset_output1.csv',index=True)

if model_answer still includes the q

In [ ]:
result['model_answer'][2].split('\n')[1]

In [ ]:
range(len(result))

In [ ]:
result_sanitised = result.copy()

In [ ]:
result_sanitised['model_answer'] = result_sanitised['model_answer'].apply(lambda x: x.split('\n')[1] if '\n' in x else x)

In [ ]:
result_sanitised[2:3]['model_answer']

In [ ]:
result_sanitised.to_csv('nx_7binstruct_baseline_output3.csv', index=True)

run eval

In [ ]:
eval_set = pd.read_csv('nx_7binstruct_baseline_output2.csv')

In [ ]:
ideal_answers = eval_set['answer']
gen_answers = eval_set['model_answer']

In [ ]:
ideal_answers

0      An eigenvector of a matrix is a nonzero vector...
1      Given the first hidden layer's description of ...
2      An inner product in a vector space provides a ...
3      The book assumes readers have a computer scien...
4      The main goals of statistical learning are to ...
                             ...                        
123    Deep learning introduces representations in te...
124    Outlier detection in machine learning is the i...
125    Long short-term memory (LSTM) in machine learn...
126    Hyperparameter tuning in machine learning invo...
127    Learned representations often result in better...
Name: answer, Length: 128, dtype: object

In [ ]:
P, R, F1 = score(gen_answers, ideal_answers, lang='en', verbose=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")